In [226]:
import pandas as pd
import numpy as np
import seaborn as sns
from dateutil.parser import parse
from datetime import datetime

In [227]:
cancer_data = pd.read_csv('cancer_data.csv')

In [228]:
enrollment= pd.read_csv('enrollment_countries_cancer.csv')

In [229]:
enrollment['NCT Number'] = enrollment['NCT Number'].str[3:].astype('int')

In [230]:
cancer_data['NCT Number'] = cancer_data['NCT Number'].str[3:].astype('int')

In [231]:
my_df= cancer_data.set_index('NCT Number').join(enrollment.set_index('NCT Number'))

In [232]:
my_df['Actual'] = my_df['Actual'].replace('Not Provided',np.NaN)

In [233]:
my_df['Actual'] = my_df['Actual'].astype('float')

In [234]:
my_df['Estimated'] = np.where(my_df['Estimated'] == 'Same as current',my_df['Actual'], my_df['Estimated'])

In [235]:
my_df['Estimated']= my_df['Estimated'].replace('Not Provided',np.NaN)

In [236]:
my_df['Estimated'] = my_df['Estimated'].astype('float')

In [237]:
my_df['EFraction']= my_df['Actual']/my_df['Estimated']

In [238]:
status_dict = {"Terminated" : 0 , "Completed" : 1}

In [239]:
my_df =my_df.replace({"Status": status_dict})

In [240]:
my_df["Phases"].unique()

array(['Not Applicable', 'Phase 1', 'Phase 3', 'Phase 2',
       'Phase 1|Phase 2', 'Phase 4', 'Phase 2|Phase 3', 'Early Phase 1'],
      dtype=object)

In [241]:
my_df['Countries']= my_df['Countries'].replace('Not Provided',np.NaN)

In [242]:
my_df["Countries"]= my_df["Countries"].str.split(',')

In [243]:
my_df["Intervention"]= my_df["Interventions"].str.split(': ').str[0]

In [245]:
my_df['Intervention'].unique()

array(['Behavioral', 'Other', 'Drug', 'Radiation', 'Procedure',
       'Dietary Supplement', 'Biological', 'Diagnostic Test', 'Device',
       'Genetic', 'Combination Product'], dtype=object)

In [246]:
intervention_dict = {'Drug': 0, 'Behavioral' : 1, 'Radiation': 2, 'Procedure':3,
       'Dietary Supplement':4, 'Biological':5, 'Diagnostic Test':6, 'Device':7,
       'Genetic':8, 'Combination Product':9, 'Other': 10}

In [247]:
my_df =my_df.replace({"Interventions": intervention_dict})

In [248]:
my_df['Disease']= 'Cancer'

In [249]:
my_df['Start Date'] = pd.to_datetime(my_df['Start Date'], errors='coerce').dt.to_period('M')

In [250]:
my_df['Completion Date'] = pd.to_datetime(my_df['Completion Date'], errors='coerce').dt.to_period('M')

In [251]:
my_df['Month']= 12 * (my_df['Completion Date'].dt.year - my_df['Start Date'].dt.year) + (my_df['Completion Date'].dt.month - my_df['Start Date'].dt.month)

In [252]:
my_df

,Rank,Title,Status,Study Results,Conditions,Interventions,Outcome Measures,Gender,Age,Phases,...,Completion Date,Locations,URL,Actual,Estimated,Countries,EFraction,Intervention,Disease,Month
NCT Number,,,,,,,,,,,,,,,,,,,,,
3254875,1,Rehabilitation After Breast Cancer,1,No Results Available,Breast Cancer,Behavioral: Individually tailored nurse naviga...,Distress|Depression|Anxiety|Health related qua...,Female,"18 Years and older (Adult, Older Adult)",Not Applicable,...,2021-03,"Rigshospitalet, Copenhagen, Denmark",https://ClinicalTrials.gov/show/NCT03254875,309.0,324.0,[Denmark],0.953704,Behavioral,Cancer,43
3736746,2,Testing Feasibility of Motivational Interviewi...,1,No Results Available,Other Cancer,Behavioral: Motivational Interviewing,Feasibility of Motivational Interviewing inter...,All,"18 Years and older (Adult, Older Adult)",Not Applicable,...,2019-08,"Dana Farber Cancer Institute, Boston, Massachu...",https://ClinicalTrials.gov/show/NCT03736746,54.0,54.0,[United States],1.000000,Behavioral,Cancer,9
3865654,3,Individualizing Surveillance Mammography for O...,1,No Results Available,Breast Cancer,Other: Communication Tool,Rate of change in intentions for mammography i...,Female,75 Years and older (Older Adult),Not Applicable,...,2021-07,"Dana-Farber Cancer Institute, Boston, Massachu...",https://ClinicalTrials.gov/show/NCT03865654,45.0,45.0,[United States],1.000000,Other,Cancer,26
2078089,4,Oxcarbazepine Plus Morphine in Patients With R...,0,No Results Available,Cancer,Drug: Morphine|Drug: Oxcarbazepine,Number of patients with adverse events as a me...,All,"18 Years and older (Adult, Older Adult)",Phase 1,...,2014-12,"Indiana Univeristy Health Hospital, Indianapol...",https://ClinicalTrials.gov/show/NCT02078089,1.0,12.0,[United States],0.083333,Drug,Cancer,9
1675622,5,Immediate-Release Oxycodone Capsules Study in ...,1,Has Results,Cancer,Drug: Oxycodone|Drug: Morphine,Numerical Rating Scale (NRS)|The Average Dose ...,All,"18 Years to 80 Years (Adult, Older Adult)",Phase 3,...,2012-07,NaN,https://ClinicalTrials.gov/show/NCT01675622,242.0,240.0,NaN,1.008333,Drug,Cancer,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1588990,9996,A Translational Study of Bevacizumab in Partic...,1,Has Results,Colorectal Neoplasms,Drug: Oxaliplatin|Drug: Capecitabine|Drug: Bev...,Association Between Neutrophil to Lymphocyte R...,All,"18 Years and older (Adult, Older Adult)",Phase 4,...,2016-09,"Canberra Hospital, Garran, Australian Capital ...",https://ClinicalTrials.gov/show/NCT01588990,128.0,150.0,[Australia],0.853333,Drug,Cancer,51
1099228,9997,Combination Targeted Radiotherapy in Neuroendo...,1,No Results Available,Neuroendocrine Tumors,Other: 131-I MIBG and 111-In pentetreotide|Oth...,Dosimetry Results|Maximum Radiation Dose,All,"18 Years and older (Adult, Older Adult)",Not Applicable,...,2015-07,"University of Iowa, Iowa City, Iowa, United St...",https://ClinicalTrials.gov/show/NCT01099228,2.0,40.0,[United States],0.050000,Other,Cancer,106
3272971,9998,Feasibility Study of the RF Ablation Catheter ...,0,No Results Available,Lung Cancer|Lung Cancer Metastatic|Lung Cancer...,Device: Radio-frequency Ablation,RFA relate AEs/SAEs|Feasibility - Ablated Tiss...,All,"21 Years to 75 Years (Adult, Older Adult)",Not Applicable,...,2019-01,The First Affiliated Hospital of Guangzhou Med...,https://ClinicalTrials.gov/show/NCT03272971,3.0,10.0,[China],0.300000,Device,Cancer,13


In [253]:
my_df.to_csv('cancer_data_cleaned.csv',index=False)